In [1]:
from fastai.vision.all import *

In [2]:
DATASET_PATH = Path('RockPaperScissors/data')
DATASET_PATH.ls()

(#5) [Path('RockPaperScissors/data/valid'),Path('RockPaperScissors/data/.DS_Store'),Path('RockPaperScissors/data/Rock-Paper-Scissors'),Path('RockPaperScissors/data/train'),Path('RockPaperScissors/data/test2')]

In [3]:
rock_train = (DATASET_PATH/'train'/'rock').ls().sorted()
paper_train = (DATASET_PATH/'train'/'paper').ls().sorted()
scissors_train = (DATASET_PATH/'train'/'scissors').ls().sorted()

#cumulate all the images of three classes as Pytorch tensors
rock_tensors = [tensor(Image.open(o)) for o in rock_train]
paper_tensors = [tensor(Image.open(o)) for o in paper_train]
scissors_tensors = [tensor(Image.open(o)) for o in scissors_train]

stacked_rock = torch.stack(rock_tensors).float()/255
stacked_paper = torch.stack(paper_tensors).float()/255
stacked_scissors = torch.stack(scissors_tensors).float()/255

stacked_rock.shape, stacked_paper.shape, stacked_scissors.shape

(torch.Size([840, 300, 300, 4]),
 torch.Size([840, 300, 300, 4]),
 torch.Size([840, 300, 300, 4]))

In [10]:
train_x = torch.cat([stacked_rock, stacked_paper, stacked_scissors]).view(-1, 300*300*4)
train_x.shape

torch.Size([2520, 360000])

In [51]:
train_x[0].shape

torch.Size([360000])

In [11]:
train_y = tensor([0] * len(rock_train) + [1] * len(paper_train) + [2] * len(scissors_train)).unsqueeze(1)
train_y.shape

torch.Size([2520, 1])

In [12]:
train_y

tensor([[0],
        [0],
        [0],
        ...,
        [2],
        [2],
        [2]])

In [26]:
train_y.shape

torch.Size([2520, 1])

In [33]:
target = train_y.view(1, -1)
target

tensor([[0, 0, 0,  ..., 2, 2, 2]])

In [13]:
dataset = list(zip(train_x, train_y))
dataset

[(tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 0.9961, 0.9961, 1.0000]), tensor([0])),
 (tensor([1.0000, 1.0000, 1.0000,  ..., 

In [14]:
rock_val = (DATASET_PATH/'valid'/'rock').ls()
paper_val = (DATASET_PATH/'valid'/'paper').ls()
scissors_val = (DATASET_PATH/'valid'/'scissors').ls()

stacked_rock_val = torch.stack([tensor(Image.open(o)) for o in rock_val])
stacked_rock_val = stacked_rock_val.float()/255
stacked_paper_val = torch.stack([tensor(Image.open(o)) for o in paper_val])
stacked_paper_val = stacked_paper_val.float()/255
stacked_scissors_val = torch.stack([tensor(Image.open(o)) for o in scissors_val])
stacked_scissors_val = stacked_scissors_val.float()/255

stacked_rock_val.shape, stacked_paper_val.shape, stacked_scissors_val.shape

(torch.Size([124, 300, 300, 4]),
 torch.Size([124, 300, 300, 4]),
 torch.Size([124, 300, 300, 4]))

In [15]:
val_x = torch.cat([stacked_rock_val, stacked_paper_val, stacked_scissors_val]).view(-1, 300*300*4)
val_y = tensor([0] * len(rock_val) + [1] * len(paper_val) + [2] * len(scissors_val)).unsqueeze(1)

val_x.shape, val_y.shape

(torch.Size([372, 360000]), torch.Size([372, 1]))

In [16]:
dataset_valid = list(zip(val_x,val_y))

In [78]:
model = nn.Linear(300*300*4, 3)
weight, bias = list(model.parameters())

weight.shape, bias.shape

(torch.Size([3, 360000]), torch.Size([3]))

### Predicting on one image

In [67]:
#prediction on one image
random_img = train_x[1000]

In [68]:
random_img.shape

torch.Size([360000])

In [84]:
x = random_img.view(1, 300*300*4)
x.shape

torch.Size([1, 360000])

In [85]:
xb = model(x)
xb

tensor([[ 0.6482,  0.1519, -0.3730]], grad_fn=<AddmmBackward>)

In [86]:
pred = torch.softmax(xb, dim = 1)
pred

tensor([[0.5079, 0.3092, 0.1829]], grad_fn=<SoftmaxBackward>)

In [66]:
criterion = nn.CrossEntropyLoss()

In [88]:
loss = criterion(pred, torch.Tensor([1]).long())
print(loss)

tensor(1.1318, grad_fn=<NllLossBackward>)


### Training pipeline

In [79]:
train_x.shape

torch.Size([2520, 360000])

In [141]:
dl = DataLoader(dataset, batch_size = 64)

In [142]:
valid_dl = DataLoader(dataset_valid, batch_size = 64)

In [116]:
xb, yb = first(dl)
xb.shape, yb.shape

(torch.Size([64, 360000]), torch.Size([64, 1]))

In [115]:
yb

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])

In [121]:
yb.long()

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])

In [125]:
yb = yb.flatten()

In [91]:
criterion = nn.CrossEntropyLoss()

In [99]:
def batch_accuracy(xb, yb):
    preds = torch.softmax(xb, dim = 1)
    predicted_class = torch.argmax(preds, dim = 1)
    correct = (predicted_class == yb)
    return correct.float().mean()

In [100]:
#to check if the function is working as expected
batch_accuracy(model(train_x[:4]), train_y[:4])

tensor(1.)

In [144]:
def calc_grad(xb, yb, model):
    preds_raw = model(xb)
    preds = torch.softmax(preds_raw, dim = 1)
    loss = criterion(preds, yb.flatten())
    print(loss)
    loss.backward()

In [118]:
preds = torch.softmax(preds_raw, dim = 1)
preds

tensor([[0.5033, 0.3458, 0.1510],
        [0.5032, 0.3456, 0.1512],
        [0.5040, 0.3433, 0.1527],
        [0.5085, 0.3372, 0.1543],
        [0.5052, 0.3423, 0.1524],
        [0.5058, 0.3446, 0.1496],
        [0.5203, 0.3362, 0.1435],
        [0.5199, 0.3373, 0.1428],
        [0.5195, 0.3410, 0.1395],
        [0.4997, 0.3574, 0.1429],
        [0.4958, 0.3593, 0.1450],
        [0.4956, 0.3627, 0.1417],
        [0.4901, 0.3682, 0.1417],
        [0.4906, 0.3693, 0.1402],
        [0.4901, 0.3676, 0.1423],
        [0.4906, 0.3624, 0.1470],
        [0.4942, 0.3589, 0.1469],
        [0.4949, 0.3588, 0.1464],
        [0.4970, 0.3569, 0.1461],
        [0.4934, 0.3610, 0.1456],
        [0.4910, 0.3648, 0.1442],
        [0.4919, 0.3667, 0.1414],
        [0.4887, 0.3725, 0.1388],
        [0.4972, 0.3637, 0.1391],
        [0.5028, 0.3567, 0.1405],
        [0.5063, 0.3513, 0.1424],
        [0.5035, 0.3554, 0.1411],
        [0.5066, 0.3560, 0.1374],
        [0.5119, 0.3459, 0.1422],
        [0.506

In [126]:
loss = criterion(preds, yb.long())

In [127]:
loss

tensor(0.9436, grad_fn=<NllLossBackward>)

In [135]:
def train_epoch(model, params, lr):
    for xb, yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad * lr
            p.grad.zero_()

In [138]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [103]:
validate_epoch(model)

0.3284

In [133]:
params = weight, bias
lr = 0.001

In [130]:
train_epoch(model, params, lr)

In [131]:
validate_epoch(model)

0.3281

In [145]:
# do for 5 epochs
for i in range(5):
    train_epoch(model, params, lr)
    print(validate_epoch(model), end = ',')

tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(0.5514, grad_fn=<NllLossBackward>)
tensor(1.4264, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBackward>)
tensor(1.5514, grad_fn=<NllLossBac

0.3229,

In [ ]:
# it's overfitting badly.